In [28]:
import os
import hashlib
from bitarray.util import ba2int

In [29]:
entropyBitSize = 128
entropyBytes = os.urandom(entropyBitSize // 8)
print("Entropy:", entropyBytes.hex())

Entropy: a0c49931528599a9cba12fea63a43146


In [30]:
from bitarray import bitarray
entropyBits = bitarray()
entropyBits.frombytes(entropyBytes)
print("Entropy bits:", entropyBits)

Entropy bits: bitarray('10100000110001001001100100110001010100101000010110011001101010011100101110100001001011111110101001100011101001000011000101000110')


In [31]:
checksumLen = entropyBitSize // 32
print("checksum size:", checksumLen)

hashBytes = hashlib.sha256(entropyBytes).digest()
print("hashed bytes:", hashBytes.hex())

hashBits = bitarray()
hashBits.frombytes(hashBytes)
checksum = hashBits[:checksumLen]
print("checksum bits", checksum)

checksum size: 4
hashed bytes: b2b7217c7e86cd297fbb63647946f640ee674893475dd2740e19afaa3e4d0a31
checksum bits bitarray('1011')


In [32]:
entropyBits.extend(checksum)
print("entropy length:", len(entropyBits))

entropy length: 132


In [33]:
indexes = list()

for idx in range(len(entropyBits) // 11):
    startIdx = idx * 11
    endIdx = startIdx + 11
    wordIdx = ba2int(entropyBits[startIdx:endIdx])
    indexes.append(wordIdx)

print(indexes)

[1286, 294, 610, 1320, 716, 1703, 372, 303, 1875, 233, 98, 1131]


In [34]:
fileObj = open("bip-0039/english.txt", "r")
words = fileObj.read().splitlines()
fileObj.close()
print("words len:", len(words))

words len: 2048


In [40]:
mnemonic = list(map(lambda idx: words[idx], indexes))
print("mnemonic:", mnemonic)

mnemonic: ['patch', 'cave', 'era', 'pink', 'flower', 'stay', 'company', 'chalk', 'tuna', 'brush', 'arrange', 'miracle']


In [41]:
password = ""
salt = "mnemonic" + password

In [43]:
mnemonicStr = " ".join(mnemonic)
seed = hashlib.pbkdf2_hmac(
    "sha512",
    mnemonicStr.encode("utf-8"),
    salt.encode("utf-8"),
    2048
)

print("seed len:", len(seed))
print("seed hex:", seed.hex())
print("priv key:", seed[0:32].hex())
print("chain co:", seed[32:64].hex())

seed len: 64
seed hex: 261ce449c0ba502ec5369affd2bb43f7d1203835c6a822fa2097a64c2cc558101cf036df88ae8367b28458d0f15373151ba9c6bd9d9b4c7a68ea00bb80556a5b
priv key: 261ce449c0ba502ec5369affd2bb43f7d1203835c6a822fa2097a64c2cc55810
chain co: 1cf036df88ae8367b28458d0f15373151ba9c6bd9d9b4c7a68ea00bb80556a5b
